In [1]:
raw_data = spark.sql("select * from datasets_228_482_diabetes_csv")
raw_data.take(1)

Out[1]: [Row(Pregnancies=6, Glucose=148, BloodPressure=72, SkinThickness=35, Insulin=0, BMI=33.6, DiabetesPedigreeFunction=0.627, Age=50, Outcome=1)]

In [2]:
raw_data.show(5)


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
 6| 148| 72| 35| 0|33.6| 0.627| 50| 1|
 1| 85| 66| 29| 0|26.6| 0.351| 31| 0|
 8| 183| 64| 0| 0|23.3| 0.672| 32| 1|
 1| 89| 66| 23| 94|28.1| 0.167| 21| 0|
 0| 137| 40| 35| 168|43.1| 2.288| 33| 1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows

In [3]:
raw_data.columns

Out[3]: ['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [4]:
raw_data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Pregnancies,768,3.8450520833333335,3.36957806269887,0,17
Glucose,768,120.89453125,31.97261819513622,0,199
BloodPressure,768,69.10546875,19.355807170644777,0,122
SkinThickness,768,20.536458333333332,15.952217567727642,0,99
Insulin,768,79.79947916666667,115.24400235133803,0,846
BMI,768,31.992578124999977,7.884160320375441,0.0,67.1
DiabetesPedigreeFunction,768,0.4718763020833327,0.331328595012775,0.078,2.42
Age,768,33.240885416666664,11.760231540678689,21,81
Outcome,768,0.3489583333333333,0.476951377242799,0,1


In [5]:
import numpy as np
from pyspark.sql.functions import *

raw_data=raw_data.withColumn("Glucose",when(raw_data.Glucose==0,np.nan).otherwise(raw_data.Glucose))
raw_data=raw_data.withColumn("BloodPressure",when(raw_data.BloodPressure==0,np.nan).otherwise(raw_data.BloodPressure))
raw_data=raw_data.withColumn("SkinThickness",when(raw_data.SkinThickness==0,np.nan).otherwise(raw_data.SkinThickness))
raw_data=raw_data.withColumn("BMI",when(raw_data.BMI==0,np.nan).otherwise(raw_data.BMI))
raw_data=raw_data.withColumn("Insulin",when(raw_data.Insulin==0,np.nan).otherwise(raw_data.Insulin))

raw_data.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
 6| 148.0| 72.0| 35.0| NaN|33.6| 0.627| 50| 1|
 1| 85.0| 66.0| 29.0| NaN|26.6| 0.351| 31| 0|
 8| 183.0| 64.0| NaN| NaN|23.3| 0.672| 32| 1|
 1| 89.0| 66.0| 23.0| 94.0|28.1| 0.167| 21| 0|
 0| 137.0| 40.0| 35.0| 168.0|43.1| 2.288| 33| 1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows

In [6]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=["Glucose","BloodPressure","SkinThickness","BMI","Insulin"],outputCols=["Glucose","BloodPressure","SkinThickness","BMI","Insulin"])
model=imputer.fit(raw_data)
raw_data=model.transform(raw_data)
raw_data.show(5)

+-----------+-------+-------------+------------------+-----------------+----+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure| SkinThickness| Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+------------------+-----------------+----+------------------------+---+-------+
 6| 148.0| 72.0| 35.0|155.5482233502538|33.6| 0.627| 50| 1|
 1| 85.0| 66.0| 29.0|155.5482233502538|26.6| 0.351| 31| 0|
 8| 183.0| 64.0|29.153419593345657|155.5482233502538|23.3| 0.672| 32| 1|
 1| 89.0| 66.0| 23.0| 94.0|28.1| 0.167| 21| 0|
 0| 137.0| 40.0| 35.0| 168.0|43.1| 2.288| 33| 1|
+-----------+-------+-------------+------------------+-----------------+----+------------------------+---+-------+
only showing top 5 rows

In [7]:
cols=raw_data.columns
cols.remove("Outcome")
# Let us import the vector assembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols,outputCol="features")
# Now let us use the transform method to transform our dataset
raw_data=assembler.transform(raw_data)
raw_data.select("features").show(truncate=False)

+-----------------------------------------------------------------------------------+
features |
+-----------------------------------------------------------------------------------+
[6.0,148.0,72.0,35.0,155.5482233502538,33.6,0.627,50.0] |
[1.0,85.0,66.0,29.0,155.5482233502538,26.6,0.351,31.0] |
[8.0,183.0,64.0,29.153419593345657,155.5482233502538,23.3,0.672,32.0] |
[1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0] |
[0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0] |
[5.0,116.0,74.0,29.153419593345657,155.5482233502538,25.6,0.201,30.0] |
[3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0] |
[10.0,115.0,72.40518417462484,29.153419593345657,155.5482233502538,35.3,0.134,29.0]|
[2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0] |
[8.0,125.0,96.0,29.153419593345657,155.5482233502538,32.45746367239099,0.232,54.0] |
[4.0,110.0,92.0,29.153419593345657,155.5482233502538,37.6,0.191,30.0] |
[10.0,168.0,74.0,29.153419593345657,155.5482233502538,38.0,0.537,34.0] |
[10.0,139.0,80.0,29.153419593345657,155.5482233502538,27.1,1.441,57.0] |
[1.0,189.0,60.0,23.0,846.0,30.1,0.398,59.0] |
[5.0,166.0,72.0,19.0,175.0,25.8,0.587,51.0] |
[7.0,100.0,72.40518417462484,29.153419593345657,155.5482233502538,30.0,0.484,32.0] |
[0.0,118.0,84.0,47.0,230.0,45.8,0.551,31.0] |
[7.0,107.0,74.0,29.153419593345657,155.5482233502538,29.6,0.254,31.0] |
[1.0,103.0,30.0,38.0,83.0,43.3,0.183,33.0] |
[1.0,115.0,70.0,30.0,96.0,34.6,0.529,32.0] |
+-----------------------------------------------------------------------------------+
only showing top 20 rows

In [8]:
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
raw_data=standardscaler.fit(raw_data).transform(raw_data)
raw_data.select("features","Scaled_features").show(5)

+--------------------+--------------------+
 features| Scaled_features|
+--------------------+--------------------+
[6.0,148.0,72.0,3...|[1.78063837321943...|
[1.0,85.0,66.0,29...|[0.29677306220323...|
[8.0,183.0,64.0,2...|[2.37418449762590...|
[1.0,89.0,66.0,23...|[0.29677306220323...|
[0.0,137.0,40.0,3...|[0.0,4.5012560836...|
+--------------------+--------------------+
only showing top 5 rows

In [9]:
train, test = raw_data.randomSplit([0.8, 0.2], seed=12345)

In [10]:
dataset_size=float(train.select("Outcome").count())
numPositives=train.select("Outcome").where('Outcome == 1').count()
per_ones=(float(numPositives)/float(dataset_size))*100
numNegatives=float(dataset_size-numPositives)
print('The number of ones are {}'.format(numPositives))
print('Percentage of ones are {}'.format(per_ones))

The number of ones are 206
Percentage of ones are 34.27620632279534

In [11]:
BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 0.6572379367720466

In [12]:
train=train.withColumn("classWeights", when(train.Outcome == 1,BalancingRatio).otherwise(1-BalancingRatio))
train.select("classWeights").show(5)

+-------------------+
 classWeights|
+-------------------+
0.34276206322795344|
0.34276206322795344|
0.34276206322795344|
0.34276206322795344|
0.34276206322795344|
+-------------------+
only showing top 5 rows

In [13]:
# Feature selection using chisquareSelector
from pyspark.ml.feature import ChiSqSelector

css = ChiSqSelector(featuresCol='Scaled_features',outputCol='Aspect',labelCol='Outcome',fpr=0.05)
train=css.fit(train).transform(train)
test=css.fit(test).transform(test)
test.select("Aspect").show(5,truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------+
Aspect |
+---------------------------------------------------------------------------------------------------------------------------------------+
[0.0,2.562758938133151,7.274924068899646,3.29885013976358,0.47047140468429116,5.367154589366346,1.3098778871870878,1.785679127775751] |
[0.0,2.8256060087109103,5.621532235058818,3.640110499049468,1.8295247783934943,5.207158111092553,0.7183201316832416,2.125808485447323] |
[0.0,2.9898854278220095,5.621532235058818,3.640110499049468,2.4699748745925287,5.8035086210221465,1.1499158410559458,2.125808485447323]|
[0.0,3.1213089631108892,5.290853868290652,4.436384670716539,1.2349874372962644,6.487129937282901,1.1046435638490186,1.870711467193644] |
[0.0,3.1213089631108892,6.6135673353633155,5.118905389288314,1.0820842307738696,5.30897405181224,0.995990098552394,2.210840824865216] |
+---------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [14]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Outcome", featuresCol="Aspect",weightCol="classWeights",maxIter=10)
model=lr.fit(train)
predict_train=model.transform(train)
predict_test=model.transform(test)
predict_test.select("Outcome","prediction").show(10)

+-------+----------+
Outcome|prediction|
+-------+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
+-------+----------+
only showing top 10 rows

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="Outcome")
predict_test.select("Outcome","prediction","probability").show(5)
print("The area under ROC for train set is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

+-------+----------+--------------------+
Outcome|prediction| probability|
+-------+----------+--------------------+
 0| 0.0|[0.93014249279729...|
 0| 0.0|[0.91059987057264...|
 0| 0.0|[0.84301258010143...|
 0| 0.0|[0.65688913922506...|
 0| 0.0|[0.85692454770533...|
+-------+----------+--------------------+
only showing top 5 rows

The area under ROC for train set is 0.838687476957108
The area under ROC for test set is 0.8447004608294926

In [16]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = ParamGridBuilder()\
    .addGrid(lr.aggregationDepth,[2,5,10])\
    .addGrid(lr.elasticNetParam,[0.0, 0.5, 1.0])\
    .addGrid(lr.fitIntercept,[False, True])\
    .addGrid(lr.maxIter,[10, 100, 1000])\
    .addGrid(lr.regParam,[0.01, 0.5, 2.0]) \
    .build()

In [17]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations
cvModel = cv.fit(train)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train=cvModel.transform(train)
predict_test=cvModel.transform(test)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_test)))

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.